Пробуем завести Ранкер по туториалу

https://github.com/catboost/catboost/blob/master/catboost/tutorials/ranking/ranking_tutorial.ipynb

In [63]:
from catboost import CatBoostRanker, Pool, MetricVisualizer
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import ndcg_score, accuracy_score

from copy import deepcopy
import numpy as np
import os
import pandas as pd

Data

In [64]:
train = pd.read_json('/kaggle/input/cl-cup-it-2023-ds/CL_Cup IT_Data_Scince___VK_/ranking_train.jsonl', lines=True)
test = pd.read_json('/kaggle/input/cl-cup-it-2023-ds/CL_Cup IT_Data_Scince___VK_/ranking_test.jsonl', lines=True)

In [65]:
def prepare_df(train, test=False):
    train = train.explode('comments')
    if not test:
        train['score'] = train['comments'].str['score']
    train['comment'] = train['comments'].str['text']
    train = train.drop(columns=['comments'])
    train['text_id'] = train.index
    return train
train = prepare_df(train)
scores = train['score'].values
test = prepare_df(test, True)
train.head(20)

,text,score,comment,text_id
0,How many summer Y Combinator fundees decided n...,0,Going back to school is not identical with giv...,0
0,How many summer Y Combinator fundees decided n...,1,There will invariably be those who don't see t...,0
0,How many summer Y Combinator fundees decided n...,2,For me school is a way to be connected to what...,0
0,How many summer Y Combinator fundees decided n...,3,I guess it really depends on how hungry you ar...,0
0,How many summer Y Combinator fundees decided n...,4,I know pollground decided to go back to school...,0
1,CBS acquires last.fm for $280m,0,It will be curious to see where this heads in ...,1
1,CBS acquires last.fm for $280m,1,Does this mean that there's now a big-name com...,1
1,CBS acquires last.fm for $280m,2,Also on BBC News: http://news.bbc.co.uk/1/low...,1
1,CBS acquires last.fm for $280m,3,I don't understand what they do that is worth ...,1
1,CBS acquires last.fm for $280m,4,sold out too cheaply. given their leadership p...,1


Embs

In [67]:
train_text_embs = np.load('/kaggle/input/cl-cup-it-2023-embs/paraphrase-MiniLM-L6-v2/paraphrase-MiniLM-L6-v2/train_text_embs.npy' ,allow_pickle=True)
test_text_embs = np.load('/kaggle/input/cl-cup-it-2023-embs/paraphrase-MiniLM-L6-v2/paraphrase-MiniLM-L6-v2/test_text_embs.npy' ,allow_pickle=True)
train_comment_embs = np.load('/kaggle/input/cl-cup-it-2023-embs/paraphrase-MiniLM-L6-v2/paraphrase-MiniLM-L6-v2/train_comment_embs.npy' ,allow_pickle=True)
test_comment_embs = np.load('/kaggle/input/cl-cup-it-2023-embs/paraphrase-MiniLM-L6-v2/paraphrase-MiniLM-L6-v2/test_comment_embs.npy' ,allow_pickle=True)

Split

In [68]:
splitter = GroupShuffleSplit(test_size=.20, n_splits=1, random_state = 9)
split = splitter.split(train_comment_embs, groups=train['text_id'])
train_inds, test_inds = next(split)

X_train = train_comment_embs[train_inds]
X_test = train_comment_embs[test_inds]
y_train = scores[train_inds]
y_test = scores[test_inds]

In [73]:
X_train = X_train[:, 0:64]
X_test = X_test[:, 0:64]

Pools

In [74]:
train_pool = Pool(
    data=X_train,
    label=y_train,
    group_id=train['text_id'].values[train_inds]
)

test_pool = Pool(
    data=X_test,
    label=y_test,
    group_id=train['text_id'].values[test_inds]
)

In [83]:
default_parameters = {
    'iterations': 10000,
#     'custom_metric': ['NDCG', 'PFound', 'AverageGain:top=10'],
    'verbose': 250,
    'random_seed': 0,
}

parameters = {}

In [84]:
def fit_model(loss_function, additional_params=None, train_pool=train_pool, test_pool=test_pool):
    parameters = deepcopy(default_parameters)
    parameters['loss_function'] = loss_function
    parameters['train_dir'] = loss_function
    
    if additional_params is not None:
        parameters.update(additional_params)
        
    model = CatBoostRanker(**parameters)
    model.fit(train_pool, eval_set=test_pool, plot=True)
    
    return model

In [85]:
# model = fit_model('RMSE',{'task_type': 'GPU', 'devices': '0:1', 'custom_metric': ['PrecisionAt:top=10', 'RecallAt:top=10', 'MAP:top=10']})
# model = fit_model('RMSE', {'task_type': 'GPU', 'devices': '0:1'})
model = fit_model('RMSE', {'task_type': 'GPU', 'devices': '0:1'}) # YetiRank YetiRankPairwise

/opt/conda/lib/python3.7/site-packages/catboost/core.py:6240: RuntimeWarning: Regression loss ('RMSE') ignores an important ranking parameter 'group_id'
  warnings.warn("Regression loss ('{}') ignores an important ranking parameter 'group_id'".format(loss_function), RuntimeWarning)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.060701
0:	learn: 1.4123779	test: 1.4124922	best: 1.4124922 (0)	total: 7.83ms	remaining: 1m 18s
250:	learn: 1.3633660	test: 1.3706075	best: 1.3706075 (250)	total: 1.74s	remaining: 1m 7s
500:	learn: 1.3552050	test: 1.3683830	best: 1.3683830 (500)	total: 3.49s	remaining: 1m 6s
750:	learn: 1.3486835	test: 1.3672833	best: 1.3672798 (749)	total: 5.59s	remaining: 1m 8s
1000:	learn: 1.3426088	test: 1.3667360	best: 1.3667204 (988)	total: 7.36s	remaining: 1m 6s
1250:	learn: 1.3368565	test: 1.3662595	best: 1.3662595 (1250)	total: 9.22s	remaining: 1m 4s
1500:	learn: 1.3314840	test: 1.3659821	best: 1.3659821 (1500)	total: 11s	remaining: 1m 2s
1750:	learn: 1.3263537	test: 1.3657721	best: 1.3657678 (1745)	total: 12.9s	remaining: 1m
2000:	learn: 1.3212706	test: 1.3656717	best: 1.3656648 (1994)	total: 14.8s	remaining: 59.1s
2250:	learn: 1.3162609	test: 1.3655942	best: 1.3655892 (2248)	total: 17.5s	remaining: 1m
2500:	learn: 1.3114302	test: 1.3655731	best: 1.3655731 (2500)	total: 

In [88]:
y_pred = model.predict(test_pool)
accuracy_score(y_test, np.round(y_pred))

0.21191692202928158